# Import libraries

In [24]:
import os
import re
import copy

import pandas as pd
import numpy as np
from sklearn import metrics

# Load and align all dataframes

In [41]:
chexpert_categories = ["No Finding", "Enlarged Cardiomediastinum", "Cardiomegaly",
                      "Lung Lesion", "Lung Opacity", "Edema", "Consolidation",
                      "Pneumonia", "Atelectasis", "Pneumothorax", "Pleural Effusion",
                      "Pleural Other", "Fracture", "Support Devices"]

# reports
val = pd.read_csv('../data_msc_project/cheXpert/input_chexpert.csv', header=None)
val.columns = ['id', 'text']

# chexbert
# df_chexbert = pd.read_csv('../data_msc_project/gpt/final_output.csv')
df_chexbert = pd.read_csv('../data_msc_project/cheXbert/chexbert_labeled_1.csv')
df_chexbert = df_chexbert.merge(
    val, how='inner', left_on='Report Impression', right_on='text'
)
df_chexbert.drop_duplicates(inplace=True)
# Drop the 's' in the beginning of the id
df_chexbert['id'] = df_chexbert['id'].astype(str).str.lstrip('s')
df_chexbert.set_index('id', inplace=True)
df_chexbert.rename(columns={'Airspace Opacity': 'Lung Opacity'}, inplace=True)
df_chexbert = df_chexbert[chexpert_categories]

# chexpert
# df_chexpert = pd.read_csv('../data_msc_project/bertdata/fixed-train-dev/bertbase-chexset-chexplabeled/output.csv')
# df_chexpert = pd.read_csv('../data_msc_project/bertdata/fixed-train-dev/bertbase-chexset-chexblabeled/output.csv')
# df_chexpert = pd.read_csv('../data_msc_project/bertdata/fixed-train-dev/bluebert-chexset-chexblabeled/output.csv')
df_chexpert = pd.read_csv('../data_msc_project/bertdata/fixed-train-dev/bluebert-balancedset-chexblabeled/output.csv')
# df_chexpert = pd.read_csv('../data_msc_project/bertdata/fixed-train-dev/bluelarge-balancedset-chexblabeled/output.csv')

# df_chexpert = pd.read_csv('../data_msc_project/bertdata/test_again.csv')
# df_chexpert = pd.read_csv('../data_msc_project/cheXpert/chexpert_labeled_1.csv')
df_chexpert = df_chexpert.merge(
    # val, how='inner', left_on='Reports', right_on='text'
    val, how='inner', left_on='Report Impression', right_on='text'
)
df_chexpert.drop_duplicates(inplace=True)
# Drop the 's' in the beginning of the id
df_chexpert['id'] = df_chexpert['id'].astype(str).str.lstrip('s')
df_chexpert.set_index('id', inplace=True)
df_chexpert.rename(columns={'Airspace Opacity': 'Lung Opacity'}, inplace=True)
df_chexpert = df_chexpert[chexpert_categories]

# visualchexbert
df_visualchexbert = pd.read_csv('../data_msc_project/VisualCheXbert/visualchexbert_labeled_1.csv')
df_visualchexbert = df_visualchexbert.merge(
    val, how='inner', left_on='Report Impression', right_on='text'
)
df_visualchexbert.drop_duplicates(inplace=True)
# Drop the 's' in the beginning of the id
df_visualchexbert['id'] = df_visualchexbert['id'].astype(str).str.lstrip('s')
df_visualchexbert.set_index('id', inplace=True)
df_visualchexbert.rename(columns={'Airspace Opacity': 'Lung Opacity'}, inplace=True)
df_visualchexbert = df_visualchexbert[chexpert_categories]

# ground truth
gs = pd.read_csv('../data_msc_project/physionet.org/files/mimic-cxr-jpg/2.1.0/mimic-cxr-2.1.0-test-set-labeled.csv', header=0, index_col=0)
gs.index.name = 'id'
gs.rename(columns={'Airspace Opacity': 'Lung Opacity'}, inplace=True)
gs = gs[chexpert_categories]

print(gs.shape)

# ensure all dataframes are aligned
gs.sort_index(inplace=True)
gs.index = gs.index.astype(str)
common_indices = gs.index.intersection(df_chexpert.index)
gs = gs.loc[common_indices]
df_chexpert = df_chexpert.loc[common_indices]
df_chexbert = df_chexbert.loc[common_indices]
df_visualchexbert = df_visualchexbert.loc[common_indices]
gs.head()

(687, 14)


,No Finding,Enlarged Cardiomediastinum,Cardiomegaly,Lung Lesion,Lung Opacity,Edema,Consolidation,Pneumonia,Atelectasis,Pneumothorax,Pleural Effusion,Pleural Other,Fracture,Support Devices
id,,,,,,,,,,,,,,
50008596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,1.0,NaN,NaN
50020371,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50022945,NaN,1.0,NaN,NaN,NaN,0.0,NaN,NaN,1.0,0.0,1.0,NaN,NaN,NaN
50031776,NaN,NaN,NaN,NaN,1.0,NaN,1.0,-1.0,-1.0,NaN,NaN,NaN,NaN,NaN
50037292,NaN,NaN,NaN,NaN,NaN,-1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN


# Evaluation functions

* Helper function to evaluate outputs in categories: (1) mentions, (2) uncertainty, (3) absence, and (4) presence.

In [42]:
def evaluate_label(tar, pred, ignore_nan=False):
    """
    Return precision, recall, f1, and prevalence for a single label.
    """
    
    if ignore_nan:
        idx = ~(np.isnan(tar) | np.isnan(pred))
        pred = pred[idx]
        tar = tar[idx]
    
    results = {
        'precision': np.nan,
        'recall': np.nan,
        'f1': np.nan,
        'positives': int(tar.sum())
    }
    
    if results['positives'] == 0:
        # return NaN if no positive labels
        return results
    
    results['precision'] = metrics.precision_score(tar, pred, zero_division=0)
    results['recall'] = metrics.recall_score(tar, pred)
    if results['precision'] + results['recall'] == 0:
        results['f1'] = 0.0
    else:
        results['f1'] = 2 * (results['precision'] * results['recall']) / (results['precision'] + results['recall'])
    
    return results
    

def get_scores(target, prediction, categories, ignore_nan=False):
    
    
    results = {}
    for i, c in enumerate(categories):
        results[c] = evaluate_label(target[:, i], prediction[:, i])
    
    # convert to dataframe
    df = pd.DataFrame.from_dict(results, orient='index')
    
    return df

def evaluate_labels(df_truth, df_label, method='mention'):
    categories = list(df_truth.columns)
    
    # create the matrix of 0s and 1s
    preds = copy.copy(df_label.values)
    targets = copy.copy(df_truth.values)
    
    if method == 'mention':
        # any mention is a 1
        preds[np.isin(preds, [-1, 0, 1])] = 1
        targets[np.isin(targets, [-1, 0, 1])] = 1

        # no mention is a 0
        preds[np.isnan(preds)] = 0
        targets[np.isnan(targets)] = 0
        
        # do not ignore NaN (which we have set to 0 anyway)
        ignore_nan=False
    elif method == 'absence':
        # successful prediction of absence
        idxNonZero = preds != 0
        idxZero = preds == 0
        preds[idxNonZero] = 0
        preds[idxZero] = 1
        
        idxNonZero = targets != 0
        idxZero = targets == 0
        targets[idxNonZero] = 0
        targets[idxZero] = 1
        
        # ignore NaN values
        ignore_nan=True
    elif method == 'presence':
        # successful prediction of presence
        idxZero = preds != 1
        idxNonZero = preds == 1
        preds[idxZero] = 0
        preds[idxNonZero] = 1

        idxZero = targets != 1
        idxNonZero = targets == 1
        targets[idxZero] = 0
        targets[idxNonZero] = 1

        # ignore NaN values
        ignore_nan=True
    elif method == 'uncertain':
        # any non-uncertain prediction is 0
        preds[preds!= -1] = 0
        targets[targets != -1] = 0
        
        # any uncertain prediction is 1
        preds[preds == -1] = 1
        targets[targets == -1] = 1
        
        # ignore NaN
        ignore_nan=True
    else:
        raise ValueError(f'Unrecognized method {method}')
        
    df = get_scores(targets, preds, categories, ignore_nan=ignore_nan)
    
    return df

# Mention 
* If the clinical finding has been mentioned in the radiology report, i.e. [1:Positive or 0:Negative or -1:Uncertain]

In [43]:
# df = evaluate_labels(gs, df_chexbert, method='presence')

# # re-order columns
# df = df[['f1', 'recall', 'precision', 'positives']]

# # round values
# for c in df.columns:
#     if 'float' in str(df.dtypes[c]):
#         df[c] = np.round(df[c], 2)

# # Replace NaN with 0
# df.fillna(0, inplace=True)

# df['f1'] = df['f1'].astype(str)
# df['recall'] = df['recall'].astype(str)
# df['precision'] = df['precision'].astype(str)
# df['f1'] = df['f1'] + ' ( ' + df['recall'] + ' , ' + df['precision'] + ' )'
# df.drop(columns=['recall', 'precision', 'positives'], inplace=True)

# # rename f1 column
# df.rename(columns={'f1': 'Presence'}, inplace=True)

# df


In [44]:
def evaluate(df_truth, df_label, method):
    df = evaluate_labels(df_truth, df_label, method=method)

    # re-order columns
    df = df[['f1', 'recall', 'precision', 'positives']]

    # round values
    for c in df.columns:
        if 'float' in str(df.dtypes[c]):
            df[c] = np.round(df[c], 2)

    # Replace NaN with 0
    df.fillna(0, inplace=True)

    # df['f1'] = df['f1'].astype(str)
    # df['recall'] = df['recall'].astype(str)
    # df['precision'] = df['precision'].astype(str)
    # df['f1'] = df['f1'] + ' ( ' + df['recall'] + ' , ' + df['precision'] + ' )'
    df.drop(columns=['recall', 'precision'], inplace=True)

    # rename f1 column
    df.rename(columns={'f1': method}, inplace=True)
    df.rename(columns={'positives': method + 'positives'}, inplace=True)

    return df

def create_table(df_truth, df_label):
    methods = ['presence', 'absence', 'uncertain']
    dfs = []
    for m in methods:
        df = evaluate(df_truth, df_label, m)
        dfs.append(df)
    df = pd.concat(dfs, axis=1)

    presenceweighted = df['presence'] * (df['presencepositives'] / (df['presencepositives'] + df['absencepositives'] + df['uncertainpositives']))
    absenceweighted = df['absence'] * (df['absencepositives'] / (df['presencepositives'] + df['absencepositives'] + df['uncertainpositives']))
    uncertainweighted = df['uncertain'] * (df['uncertainpositives'] / (df['presencepositives'] + df['absencepositives'] + df['uncertainpositives']))

    # Add weighted-F1 column
    df['Weighted-F1'] = np.round(presenceweighted + absenceweighted + uncertainweighted, 2)

    # Drop the positives columns
    df.drop(columns=['presencepositives', 'absencepositives', 'uncertainpositives'], inplace=True)

    # Print average of Weighted-F1
    print('Average Weighted F1:', np.round(np.mean(df['Weighted-F1']), 2))
    
    return df

df = create_table(gs, df_chexbert)
df.index.name = 'CheXbert'
# df.to_latex('table.tex')
df

Average Weighted F1: 0.89


,presence,absence,uncertain,Weighted-F1
CheXbert,,,,
No Finding,0.53,0.00,0.00,0.53
Enlarged Cardiomediastinum,0.85,0.93,0.91,0.89
Cardiomegaly,0.98,0.96,0.92,0.97
Lung Lesion,0.95,0.86,0.88,0.94
Lung Opacity,0.95,0.89,0.00,0.94
Edema,0.98,0.98,0.99,0.98
Consolidation,0.99,1.00,0.98,0.99
Pneumonia,0.95,0.99,0.95,0.97
Atelectasis,0.98,0.89,0.95,0.97


In [45]:
cx = create_table(gs, df_chexpert)
cx.index.name = 'OUR'
cx.to_latex('table.tex')
cx

Average Weighted F1: 0.88


,presence,absence,uncertain,Weighted-F1
OUR,,,,
No Finding,0.52,0.00,0.00,0.52
Enlarged Cardiomediastinum,0.79,0.91,0.86,0.84
Cardiomegaly,0.98,0.96,0.91,0.96
Lung Lesion,0.94,0.86,0.88,0.93
Lung Opacity,0.92,0.83,0.00,0.91
Edema,0.97,0.97,0.94,0.97
Consolidation,0.97,1.00,0.94,0.97
Pneumonia,0.93,0.99,0.94,0.96
Atelectasis,0.98,0.75,0.94,0.97


In [21]:
df = evaluate_labels(gs, df_chexbert, method='mention')
df.columns = pd.MultiIndex.from_tuples([('CheXbert', c) for c in df.columns])

cx = evaluate_labels(gs, df_chexpert, method='mention')
cx.columns = pd.MultiIndex.from_tuples([('Benchmark', c) for c in cx.columns])

df = df.merge(cx, how='inner', left_index=True, right_index=True)
# df.columns.swaplabel(0, 1, axis=1, inplace=True)
df.columns = df.columns.reorder_levels([1, 0])

# re-order columns
df = df[['precision', 'recall', 'f1', 'positives']]

# round values
for c in df.columns:
    if 'float' in str(df.dtypes[c]):
        df[c] = np.round(df[c], 3)

# drop the unecessary final column
# df.drop(('positives', 'CheXpert'), axis=1, inplace=True)
# df.drop(('positives', 'CheXbert'), axis=1, inplace=True)

# output to latex
df.index.name = 'Mention'

# df.to_latex('mention.tex')

df

precision             recall                 f1  \
                            CheXbert Benchmark CheXbert Benchmark CheXbert   
Mention                                                                      
No Finding                     0.391     0.385    0.833     0.833    0.532   
Enlarged Cardiomediastinum     0.928     0.817    0.904     0.904    0.916   
Cardiomegaly                   0.982     0.961    0.989     0.968    0.986   
Lung Lesion                    0.984     0.919    0.955     0.864    0.969   
Lung Opacity                   0.947     0.867    0.932     0.885    0.940   
Edema                          0.996     0.993    0.996     0.996    0.996   
Consolidation                  1.000     1.000    1.000     0.981    1.000   
Pneumonia                      0.991     0.975    0.996     0.991    0.994   
Atelectasis                    1.000     0.996    0.996     0.992    0.998   
Pneumothorax                   1.000     1.000    0.987     0.983    0.994   
Pleural Effusion               0.997     0.992    0.995     0.984    0.996   
Pleural Other                  0.465     0.465    0.741     0.741    0.571   
Fracture                       0.980     0.889    0.960     0.960    0.970   
Support Devices                0.958     0.936    0.983     0.953    0.970   

                                     positives            
                           Benchmark  CheXbert Benchmark  
Mention                                                   
No Finding                     0.526        30        30  
Enlarged Cardiomediastinum     0.858       114       114  
Cardiomegaly                   0.964       279       279  
Lung Lesion                    0.891        66        66  
Lung Opacity                   0.876       192       192  
Edema                          0.995       283       283  
Consolidation                  0.990       106       106  
Pneumonia                      0.983       233       233  
Atelectasis                    0.994       242       242  
Pneumothorax                   0.992       238       238  
Pleural Effusion               0.988       379       379  
Pleural Other                  0.571        27        27  
Fracture                       0.923        50        50  
Support Devices                0.944       232       232

# Uncertain

* If the clinical finding is uncertain, i.e. [-1:Uncertain]

In [5]:
df = evaluate_labels(gs, df_chexbert, method='uncertain')
df.columns = pd.MultiIndex.from_tuples([('CheXbert', c) for c in df.columns])

cx = evaluate_labels(gs, df_chexpert, method='uncertain')
cx.columns = pd.MultiIndex.from_tuples([('Benchmark', c) for c in cx.columns])

df = df.merge(cx, how='inner', left_index=True, right_index=True)
# df.columns.swaplabel(0, 1, axis=1, inplace=True)
df.columns = df.columns.reorder_levels([1, 0])

# re-order columns
df = df[['precision', 'recall', 'f1', 'positives']]

# round values
for c in df.columns:
    if 'float' in str(df.dtypes[c]):
        df[c] = np.round(df[c], 3)

# drop the unecessary final column
# df.drop(('positives', 'CheXpert'), axis=1, inplace=True)
# df.drop(('positives', 'CheXbert'), axis=1, inplace=True)

# output to latex
df.index.name = 'Uncertainty'

# df.to_latex('uncertainty.tex')

df

precision             recall                 f1  \
                            CheXbert Benchmark CheXbert Benchmark CheXbert   
Uncertainty                                                                  
No Finding                       NaN       NaN      NaN       NaN      NaN   
Enlarged Cardiomediastinum     0.897     0.778    0.921     0.921    0.909   
Cardiomegaly                   0.923     0.744    0.923     0.821    0.923   
Lung Lesion                    1.000     0.750    0.778     0.667    0.875   
Lung Opacity                     NaN       NaN      NaN       NaN      NaN   
Edema                          1.000     0.878    0.976     0.878    0.988   
Consolidation                  1.000     0.958    0.962     0.885    0.980   
Pneumonia                      0.934     0.854    0.966     0.932    0.950   
Atelectasis                    0.950     0.923    0.950     0.900    0.950   
Pneumothorax                   0.900     0.667    0.900     0.800    0.900   
Pleural Effusion               0.846     0.586    0.957     0.739    0.898   
Pleural Other                    NaN       NaN      NaN       NaN      NaN   
Fracture                       0.667     1.000    1.000     1.000    0.800   
Support Devices                  NaN       NaN      NaN       NaN      NaN   

                                     positives            
                           Benchmark  CheXbert Benchmark  
Uncertainty                                               
No Finding                       NaN         0         0  
Enlarged Cardiomediastinum     0.843        38        38  
Cardiomegaly                   0.780        39        39  
Lung Lesion                    0.706         9         9  
Lung Opacity                     NaN         0         0  
Edema                          0.878        41        41  
Consolidation                  0.920        26        26  
Pneumonia                      0.891        88        88  
Atelectasis                    0.911        40        40  
Pneumothorax                   0.727        10        10  
Pleural Effusion               0.654        23        23  
Pleural Other                    NaN         0         0  
Fracture                       1.000         2         2  
Support Devices                  NaN         0         0

# Absence

* If the clinical finding is absent in the radiology report, i.e. [0:Negative]

In [6]:
df = evaluate_labels(gs, df_chexbert, method='absence')
df.columns = pd.MultiIndex.from_tuples([('CheXbert', c) for c in df.columns])

cx = evaluate_labels(gs, df_chexpert, method='absence')
cx.columns = pd.MultiIndex.from_tuples([('Benchmark', c) for c in cx.columns])

df = df.merge(cx, how='inner', left_index=True, right_index=True)
# df.columns.swaplabel(0, 1, axis=1, inplace=True)
df.columns = df.columns.reorder_levels([1, 0])

# re-order columns
df = df[['precision', 'recall', 'f1', 'positives']]

# round values
for c in df.columns:
    if 'float' in str(df.dtypes[c]):
        df[c] = np.round(df[c], 3)

# drop the unecessary final column (redundant)
# df.drop(('positives', 'CheXpert'), axis=1, inplace=True)
# df.drop(('positives', 'CheXbert'), axis=1, inplace=True)

# output to latex
df.index.name = 'Absence'

# df.to_latex('absence.tex')

df

precision             recall                 f1  \
                            CheXbert Benchmark CheXbert Benchmark CheXbert   
Absence                                                                      
No Finding                       NaN       NaN      NaN       NaN      NaN   
Enlarged Cardiomediastinum     1.000     0.963    0.867     0.867    0.929   
Cardiomegaly                   0.956     0.921    0.966     0.921    0.961   
Lung Lesion                    1.000     1.000    0.750     0.500    0.857   
Lung Opacity                   0.909     0.762    0.870     0.696    0.889   
Edema                          0.990     0.927    0.972     0.944    0.981   
Consolidation                  1.000     0.923    1.000     1.000    1.000   
Pneumonia                      1.000     0.966    0.989     0.944    0.994   
Atelectasis                    1.000     0.667    0.800     0.400    0.889   
Pneumothorax                   0.995     0.984    0.974     0.953    0.984   
Pleural Effusion               0.976     0.916    0.964     0.916    0.970   
Pleural Other                  0.000     0.000    0.000     0.000    0.000   
Fracture                       1.000     1.000    0.750     0.625    0.857   
Support Devices                1.000     0.500    0.600     0.400    0.750   

                                     positives            
                           Benchmark  CheXbert Benchmark  
Absence                                                   
No Finding                       NaN         0         0  
Enlarged Cardiomediastinum     0.912        30        30  
Cardiomegaly                   0.921        89        89  
Lung Lesion                    0.667         4         4  
Lung Opacity                   0.727        23        23  
Edema                          0.935       107       107  
Consolidation                  0.960        24        24  
Pneumonia                      0.955        90        90  
Atelectasis                    0.500         5         5  
Pneumothorax                   0.968       190       190  
Pleural Effusion               0.916        83        83  
Pleural Other                  0.000         2         2  
Fracture                       0.769         8         8  
Support Devices                0.444         5         5

# Presence

* If the clinical finding is present in the radiology report, i.e. [1:Positive]

In [7]:
df = evaluate_labels(gs, df_chexbert, method='presence')
df.columns = pd.MultiIndex.from_tuples([('CheXbert', c) for c in df.columns])

cx = evaluate_labels(gs, df_chexpert, method='presence')
cx.columns = pd.MultiIndex.from_tuples([('Benchmark', c) for c in cx.columns])

vx = evaluate_labels(gs, df_visualchexbert, method='presence')
vx.columns = pd.MultiIndex.from_tuples([('VisualCheXbert', c) for c in vx.columns])

df = df.merge(cx, how='inner', left_index=True, right_index=True)
df = df.merge(vx, how='inner', left_index=True, right_index=True)
# df.columns.swaplabel(0, 1, axis=1, inplace=True)
df.columns = df.columns.reorder_levels([1, 0])

# re-order columns
df = df[['precision', 'recall', 'f1', 'positives']]

# round values
for c in df.columns:
    if 'float' in str(df.dtypes[c]):
        df[c] = np.round(df[c], 3)

# drop the unecessary final column (redundant)
# df.drop(('positives', 'CheXpert'), axis=1, inplace=True)
# df.drop(('positives', 'CheXbert'), axis=1, inplace=True)

# output to latex
df.index.name = 'Presence'

# df.to_latex('presence.tex')

df

precision                            recall  \
                            CheXbert Benchmark VisualCheXbert CheXbert   
Presence                                                                 
No Finding                     0.391     0.385          0.225    0.833   
Enlarged Cardiomediastinum     0.848     0.630          0.071    0.848   
Cardiomegaly                   0.980     0.980          0.406    0.987   
Lung Lesion                    0.944     0.885          0.237    0.962   
Lung Opacity                   0.952     0.874          0.278    0.941   
Edema                          0.971     0.940          0.446    0.993   
Consolidation                  0.982     0.963          0.141    1.000   
Pneumonia                      0.963     0.887          0.202    0.945   
Atelectasis                    0.985     0.965          0.429    0.985   
Pneumothorax                   0.949     0.895          0.456    0.974   
Pleural Effusion               0.993     0.970          0.812    0.982   
Pleural Other                  0.417     0.417          0.086    0.600   
Fracture                       0.950     0.809          0.056    0.950   
Support Devices                0.949     0.935          0.723    0.982   

                                                          f1            \
                           Benchmark VisualCheXbert CheXbert Benchmark   
Presence                                                                 
No Finding                     0.833          0.833    0.532     0.526   
Enlarged Cardiomediastinum     0.739          0.652    0.848     0.680   
Cardiomegaly                   0.967          0.934    0.983     0.973   
Lung Lesion                    0.868          0.604    0.953     0.876   
Lung Opacity                   0.905          0.787    0.946     0.890   
Edema                          0.933          0.941    0.982     0.937   
Consolidation                  0.929          0.857    0.991     0.945   
Pneumonia                      0.855          0.782    0.954     0.870   
Atelectasis                    0.975          0.792    0.985     0.970   
Pneumothorax                   0.895          0.816    0.961     0.895   
Pleural Effusion               0.938          0.905    0.987     0.953   
Pleural Other                  0.600          0.520    0.492     0.492   
Fracture                       0.950          0.400    0.950     0.874   
Support Devices                0.956          0.965    0.965     0.946   

                                          positives                           
                           VisualCheXbert  CheXbert Benchmark VisualCheXbert  
Presence                                                                      
No Finding                          0.355        30        30             30  
Enlarged Cardiomediastinum          0.129        46        46             46  
Cardiomegaly                        0.566       151       151            151  
Lung Lesion                         0.340        53        53             53  
Lung Opacity                        0.410       169       169            169  
Edema                               0.605       135       135            135  
Consolidation                       0.242        56        56             56  
Pneumonia                           0.321        55        55             55  
Atelectasis                         0.556       197       197            197  
Pneumothorax                        0.585        38        38             38  
Pleural Effusion                    0.856       273       273            273  
Pleural Other                       0.147        25        25             25  
Fracture                            0.098        40        40             40  
Support Devices                     0.826       227       227            227